<a href="https://colab.research.google.com/github/lixinso/sample-intro-to-pytorch/blob/master/sample_pytorch_customized_data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html